In [1]:
import os

In [2]:
%pwd

'd:\\project\\Face-to-BMI-Prediction\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\project\\Face-to-BMI-Prediction'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list
    train_yaml: Path


In [6]:
from BMI.constants import *
from BMI.utils.common import read_yaml, create_directories
import tensorflow as tf

[2024-02-09 00:33:48,387: WARNING: module_wrapper: From d:\project\Face-to-BMI-Prediction\venv\lib\site-packages\keras\src\losses.py:2976: The name tf.losses.sparse_softmax_cross_entropy is deprecated. Please use tf.compat.v1.losses.sparse_softmax_cross_entropy instead.
]


In [7]:
# import os
# from pathlib import Path
# from cnnClassifier.utils.common import read_yaml, create_directories
# from cnnClassifier.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH
# from cnnClassifier.training_config import TrainingConfig  # Assuming you have a separate module for TrainingConfig

class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params

        # Define paths for training image and metadata
        training_data = os.path.join(
            self.config.data_ingestion.unzip_dir, "dataset", "Train"
        )
        train_yaml = os.path.join(
            self.config.data_ingestion.unzip_dir, "dataset", "Train_metadata.yaml"
        )

        # Create directories if they don't exist
        create_directories([
            Path(training.root_dir),
            Path(training.trained_model_path).parent,  # Create parent directory for trained model path
            Path(prepare_base_model.updated_base_model_path).parent,  # Create parent directory for updated base model path
        ])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_image=Path(training_data),
            training_metadata=Path(train_yaml),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE
        )
        return training_config


In [8]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time 
import numpy as np
from sklearn.model_selection import train_test_split
import yaml

In [9]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config
    
    def load_metadata(self):
        with open(self.config.training_metadata, 'r') as file:
            self.metadata = yaml.safe_load(file)

    def load_images(self):
        image_files = os.listdir(self.config.training_data)
        images = []
        for image_file in image_files:
            image_path = os.path.join(self.config.training_data, image_file)
            image = tf.keras.preprocessing.image.load_img(image_path, target_size=tuple(self.config.params_image_size[:-1]))
            image = tf.keras.preprocessing.image.img_to_array(image)
            images.append(image)
        self.images = np.array(images)

    def preprocess_metadata(self):
        # Normalize metadata if necessary
        metadata_array = np.array([[sample['age'], sample['bmi'], sample['sex']] for sample in self.metadata])
        metadata_mean = metadata_array.mean(axis=0)
        metadata_std = metadata_array.std(axis=0)
        normalized_metadata = (metadata_array - metadata_mean) / metadata_std
        return normalized_metadata


    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )

    def train_valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.20
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator

        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )

    
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)



    
    def train(self):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )

In [10]:
try:
    config = ConfigurationManager()
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train()
    
except Exception as e:
    raise e

[2024-02-09 00:33:49,966: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-02-09 00:33:49,972: INFO: common: yaml file: params.yaml loaded successfully]
[2024-02-09 00:33:49,974: INFO: common: created directory at: artifacts\training]
[2024-02-09 00:33:49,976: INFO: common: created directory at: artifacts\training]
[2024-02-09 00:33:49,978: INFO: common: created directory at: artifacts\prepare_base_model]


TypeError: TrainingConfig.__init__() got an unexpected keyword argument 'training_image'